In [7]:
from scipy.stats import multivariate_normal
from random import randint
import numpy as np

In [8]:
x, y = [], []
file = open("2gaussian.txt", 'r')
k = 2
for line in file:
    x.append(float(line.split()[0]))
    y.append(float(line.split()[1]))

In [9]:
u = [[2,2], [6,3]]
cov_mat = [[[1,0],[0,2]], [[1,0.9], [0.9,1]]]
weights = {0: 0.48733333333333334, 1: 0.5126666666666667}
pi_0 = {}
pi_1 = {}

In [15]:
def probablity(data_point, u, cov_mat):
    var = multivariate_normal(mean=u, cov=cov_mat)
    probability = var.pdf([data_point[0], data_point[1]])
    return probability

In [16]:
def expectation():
    for i in range(0, len(x)):
        data_point = (x[i], y[i])
        probabilities = []
        for j in range(0, k):
            prob = probablity(data_point, u[j], cov_mat[j])
            if j == 0:
                pi_0[data_point] = prob * weights[0]
            else:
                pi_1[data_point] = prob * weights[1]
            probabilities.append((prob, j))
        denom = 0
        for prob in probabilities:
            label = prob[1]
            denom += prob[0] * weights[label]

        pi_0[data_point] /= denom
        pi_1[data_point] /= denom
        

In [12]:
def maximization():
    weights[0] = sum(list(pi_0.values())) / len(x)
    weights[1] = sum(list(pi_1.values())) / len(x)
    sum_u0_x = 0
    sum_u0_y = 0
    sum_u1_x = 0
    sum_u1_y = 0
    for i in range(0, len(x)):
        data_point = (x[i], y[i])
        p0 = pi_0[data_point]
        p1 = pi_1[data_point]
        sum_u0_x += (p0 * x[i])
        sum_u0_y += (p0 * y[i])
        sum_u1_x += (p1 * x[i])
        sum_u1_y += (p1 * y[i])
    sum_u0_x /= sum(list(pi_0.values()))
    sum_u0_y /= sum(list(pi_0.values()))
    sum_u1_x /= sum(list(pi_1.values()))
    sum_u1_y /= sum(list(pi_1.values()))
    u[0][0] = sum_u0_x
    u[0][1] = sum_u0_y
    u[1][0] = sum_u1_x
    u[1][1] = sum_u1_y
    update_cov_mat()

In [13]:
def update_cov_mat():
    final_cov_mat0 = [[0,0], [0,0]]
    final_cov_mat1 = [[0,0], [0,0]]
    for i in range(len(x)):
        data_point = (x[i], y[i])
        prob0 = pi_0[data_point]
        prob1 = pi_1[data_point]
        
        factor01 = [data_point[0] - u[0][0], data_point[1] - u[0][1]]
        factor01 = np.array(factor01).reshape(1,-1)
        factor02 = np.dot(factor01.reshape(-1,1), factor01)
        factor03 = prob0 * factor02
        
        for m in range(k):
            for n in range(k):
                final_cov_mat0[m][n] += factor03[m][n]
        
        factor11 = [data_point[0] - u[1][0], data_point[1] - u[1][1]]
        factor11 = np.array(factor11).reshape(1,-1)
        factor12 = np.dot(factor11.reshape(-1,1), factor11)
        factor13 = prob1 * factor12
        
        for m in range(k):
            for n in range(k):
                final_cov_mat1[m][n] += factor13[m][n]
                
    for m in range(k):
        for n in range(k):
            final_cov_mat0[m][n] = final_cov_mat0[m][n] / sum(list(pi_0.values()))
    
    for m in range(k):
        for n in range(k):
            final_cov_mat1[m][n] = final_cov_mat1[m][n] / sum(list(pi_1.values()))
    
    cov_mat[0] = final_cov_mat0
    cov_mat[1] = final_cov_mat1

In [14]:
def repeat_untill_convergence():
    i = 0
    while(i <= 26):
        print("Iteration: " + str(i))
        expectation()
        print("Expectation done")
        maximization()
        print("Maximization done")
        print("Mean 1: ", end = " ")
        print(u[0])
        print("Mean 2: ", end = " ")
        print(u[1])
        print("Covariance 1: ", end = " ")
        print(cov_mat[0])
        print("Covariance 2: ", end = " ")
        print(cov_mat[1])
        print("Weights: ", end = " ")
        print(weights)
        print("=============================================")
        i += 1

In [174]:
repeat_untill_convergence()

Iteration: 0
Expectation done
Maximization done
[2.868938552786409, 3.069212492675539]
[6.928626743983444, 3.942774137525465]
[[0.8398859825126438, 0.04710994849518981], [0.04710994849518981, 3.054032553967042]]
[[1.1515339886138152, 0.5792466055083737], [0.5792466055083737, 1.0596885803419758]]
{0: 0.3106219195930774, 1: 0.6893780804069234}
Iteration: 1
Expectation done
Maximization done
[2.90906421007665, 3.0660489065491436]
[6.960640448171374, 3.9551929606253466]
[[0.8861286162292746, 0.04627210520152532], [0.04627210520152532, 3.0105651477072133]]
[[1.0783713442719545, 0.5519768809926096], [0.5519768809926096, 1.042793925891946]]
{0: 0.3191453812294417, 1: 0.6808546187705568}
Iteration: 2
Expectation done
Maximization done
[2.93658658007129, 3.0613610070872737]
[6.979715109576201, 3.9645332220784373]
[[0.9224203985857908, 0.040739045574489936], [0.040739045574489936, 2.9849408033861278]]
[[1.0380317705062352, 0.5324648732236875], [0.5324648732236875, 1.0290134889689415]]
{0: 0.3245

Expectation done
Maximization done
[2.994098861346111, 3.0521005214645585]
[7.013131192871419, 3.9831244020578263]
[[1.0101797885522863, 0.027197331362791854], [0.027197331362791854, 2.9378485667316276]]
[[0.9747887692560276, 0.49748724617204315], [0.49748724617204315, 1.0011569028353837]]
{0: 0.33479018748682216, 1: 0.6652098125131799}
Iteration: 22
Expectation done
Maximization done
[2.9941095617494415, 3.0520992481826235]
[7.013136750603961, 3.9831275779038564]
[[1.0101974689725313, 0.027195402236251016], [0.027195402236251016, 2.937840256900652]]
[[0.9747790818188721, 0.4974817476935718], [0.4974817476935718, 1.0011522583805696]]
{0: 0.33479199873314985, 1: 0.6652080012668503}
Iteration: 23
Expectation done
Maximization done
[2.9941167879247943, 3.052098388429136]
[7.013140503724454, 3.9831297225415994]
[[1.010209409134885, 0.02719409965592203], [0.02719409965592203, 2.937834645189724]]
[[0.9747725400752414, 0.49747803468430446], [0.49747803468430446, 1.0011491220336641]]
{0: 0.334